# English Fiction 2gram

In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

sc=SparkContext()
spark = SparkSession.builder.getOrCreate()

In [ ]:
# import text file
ngram = spark.sparkContext.sequenceFile('s3://datasets.elasticmapreduce/ngrams/books/20090715/eng-fiction-all/2gram/data')

In [ ]:
a = ngram.first()

In [ ]:
type(a)

In [ ]:
ngram.take(5)

### Pre-processing

In [79]:
import sys
import os,datetime, re
from pyspark.sql import Row
from pyspark.sql.types import *

In [84]:
def unpack_to_row(tp):
    (num, line) = tp
    string, y, n1, n2, n3 = re.split(r'[\t]', line)
    #return [num, string, int(y), int(n1), int(n2), int(n3)]
    return Row(index=num, two_gram=string, 
               year=int(y), 
               match_count=int(n1), page_count=int(n2), volume_count=int(n3))
               # year=datetime.datetime.strptime(y, '%Y')

In [76]:
rdd = ngram.map(lambda ls: unpack_to_row(ls))

In [87]:
rdd.take(5)

[Row(index=1, match_count=1, page_count=1, two_gram='! 00000', volume_count=1, year=1796),
 Row(index=2, match_count=1, page_count=1, two_gram='! 00000', volume_count=1, year=1832),
 Row(index=3, match_count=1, page_count=1, two_gram='! 00000', volume_count=1, year=1838),
 Row(index=4, match_count=1, page_count=1, two_gram='! 00000', volume_count=1, year=1840),
 Row(index=5, match_count=1, page_count=1, two_gram='! 00000', volume_count=1, year=1841)]

In [85]:
schema = StructType([
    StructField('index', IntegerType(), True),
    StructField('two_gram', StringType(), True),
    StructField('year', IntegerType(), True),
    StructField('match_count', IntegerType(), True),
    StructField('page_count', IntegerType(), True),
    StructField('volume_count', IntegerType(), True)
])

In [86]:
# Create data frame
df = spark.createDataFrame(rdd, schema)
print(df.schema)
df.show()

StructType(List(StructField(index,IntegerType,true),StructField(two_gram,StringType,true),StructField(year,IntegerType,true),StructField(match_count,IntegerType,true),StructField(page_count,IntegerType,true),StructField(volume_count,IntegerType,true)))
+-----+--------+----+-----------+----------+------------+
|index|two_gram|year|match_count|page_count|volume_count|
+-----+--------+----+-----------+----------+------------+
|    1| ! 00000|1796|          1|         1|           1|
|    2| ! 00000|1832|          1|         1|           1|
|    3| ! 00000|1838|          1|         1|           1|
|    4| ! 00000|1840|          1|         1|           1|
|    5| ! 00000|1841|          1|         1|           1|
|    6| ! 00000|1881|          1|         1|           1|
|    7| ! 00000|1883|          1|         1|           1|
|    8| ! 00000|1892|          2|         2|           2|
|    9| ! 00000|1894|          1|         1|           1|
|   10| ! 00000|1897|          1|         1|       